<a href="https://colab.research.google.com/github/Vedarutvija/chatbot/blob/main/Push_to_Hub_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.1 MB/s eta 0:00:00


In [4]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [5]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00


In [11]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("glue", "cola")

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [7]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

(…)bert-base-cased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

(…)o/bert-base-cased/resolve/main/vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

(…)t-base-cased/resolve/main/tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [8]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation =True)
tokenized_datasets =raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import numpy as np
from evaluate import load
metric = load("glue", "cola")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis =-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments
args = TrainingArguments(
"bert-fine-tuned-cola",
evaluation_strategy ="epoch",
save_strategy="epoch",
learning_rate=2e-5,
num_train_epochs=3,
weight_decay=0.01,
push_to_hub=True,
hub_model_id = "veda",
)


In [14]:
from transformers import Trainer
trainer = Trainer(
model,
args,
train_dataset=tokenized_datasets["train"],
eval_dataset=tokenized_datasets["validation"],
compute_metrics=compute_metrics,
tokenizer=tokenizer,
)
trainer.train()



You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.457700,0.434601,0.526242
2,0.328400,0.648446,0.580551
3,0.196600,0.776220,0.582515


TrainOutput(global_step=3207, training_loss=0.33764665923015996, metrics={'train_runtime': 317.1544, 'train_samples_per_second': 80.885, 'train_steps_per_second': 10.112, 'total_flos': 243897267764580.0, 'train_loss': 0.33764665923015996, 'epoch': 3.0})

In [15]:
trainer.push_to_hub("End of training")

'https://huggingface.co/Vedarutvija/veda/tree/main/'

In [21]:
repo_name = "bert-fine-tuned-cola"
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/Vedarutvija/bert-fine-tuned-cola/commit/e90dd43a8f30ace90184396e5c687795cbc5be56', commit_message='Upload tokenizer', commit_description='', oid='e90dd43a8f30ace90184396e5c687795cbc5be56', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
label_names = raw_datasets["train"].features["label"].names
label_names

['unacceptable', 'acceptable']

In [25]:
model.config.id2label = {str(i): lbl for i, lbl in enumerate(label_names)}
model.config.label2id = {lbl: str(i) for i, lbl in enumerate(label_names)}

In [26]:
repo_name = "bert-fine-tuned-cola"
model.config.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/Vedarutvija/bert-fine-tuned-cola/commit/93f0adad79e450859d2f084602d98d62c06163fd', commit_message='Upload config', commit_description='', oid='93f0adad79e450859d2f084602d98d62c06163fd', pr_url=None, pr_revision=None, pr_num=None)